In [ ]:
import tools
import visualization_2D as vis2D
import visualization_3D as vis3D
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot_2samples
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt
import random as rd
import plotly.graph_objects as go
import plotly.express as px
import os

# I- TRN download

In [ ]:
#Sun et al., 2019
#https://wiley.figshare.com/ndownloader/files/14670236
Yeast_TRN = pd.read_csv('../results/Yeast_TRN.csv', sep='\t')
Yeast_TRN

# II- Targets lists creation

## 1) All targets

In [ ]:
list_TF = Yeast_TRN.TF.unique()
#len(list_TF) = 186

for TF in list_TF :
    
    TG = Yeast_TRN[Yeast_TRN.TF == TF]
    TG = TG.drop(["TF", "Distance"], axis = 1)
    TG.to_csv("../results/TF_target_TRN/" + str(TF) + "_" + str(len(TG)) + "_targets.csv", 
              index = False, 
              columns = ["TG", "Direction"])

## 2) Targets filtered by type of regulation (+/-)

In [ ]:
list_TF = Yeast_TRN.TF.unique()
#len(list_TF) = 186

for TF in list_TF :
    
    TG = Yeast_TRN[Yeast_TRN.TF == TF]
    TG = TG.drop(["TF", "Distance"], axis = 1)
    TG = TG[TG.Direction == "+"]
    TG.to_csv("../results/TF_target_TRN_pos/" + str(TF) + "_" + str(len(TG)) + "_pos_targets.csv", 
              index = False, 
              columns = ["TG", "Direction"])

for TF in list_TF :
    
    TG = Yeast_TRN[Yeast_TRN.TF == TF]
    TG = TG.drop(["TF", "Distance"], axis = 1)
    TG = TG[TG.Direction == "-"]
    TG.to_csv("../results/TF_target_TRN_neg/" + str(TF) + "_" + str(len(TG)) + "_neg_targets.csv", 
              index = False, 
              columns = ["TG", "Direction"])

## 3) Activated targets filtered by GO terms

In [ ]:
files_names = os.listdir("../results/TF_target_TRN_pos")

for genes in files_names:
    genes_list = pd.read_csv('../results/TF_target_TRN_pos/' + genes, sep=',', header = [0])
        
    if len(genes_list.TG) >= 50 :
        genes_list.to_csv("../results/TF_target_TRN_pos_for_SGD_GOterms/" + genes, 
                          index = False, header = False, columns = ["TG"])


# III- Repartition and 3D distances histograms

## 1) Targets repartition on the chromosomes

In [ ]:
def chrom_repartition_hist(genes_list):
    
    sql_query = \
"""SELECT Primary_SGDID, Feature_name, Start_coordinate, Stop_coordinate, Chromosome, Strand
FROM SGD_features
ORDER BY Start_coordinate
"""
    
    loci = tools.get_locus_info("../SCERE.db", sql_query)
    loci = loci.assign(FT_target = loci.Feature_name.isin(genes_list))
       
    loci = loci[loci.FT_target == True].drop(["FT_target"], axis = 1)
       
    fig = px.histogram(loci, x="Chromosome", nbins=30, range_x=[1, 17], color_discrete_sequence=['#A0E8AF'])
    fig.update_layout(plot_bgcolor = "white",
                      xaxis_showgrid = False,
                      yaxis_showgrid = False, 
                      showlegend = True)
       
    return fig

## 3) Results

### a) All targets together

In [ ]:
adjacency_matrix.isnull().sum().sum()

In [ ]:
((9185*9185) - 9185)/2

In [ ]:
42186705-42177520

In [ ]:
adjacency_matrix = pd.read_parquet("../dashboard/static/adjacency_matrix.parquet.gzip", engine='pyarrow')
files_names = os.listdir("../results/TF_target_TRN")

#removed mitochondrial targets genes :
#Q0050, Q0080 and Q0120 from YIR018W_451_targets
#Q0115 and Q0130 from YJL127C_1176_targets
#Q0080 and Q0297 from 'YEL009C_303_targets.csv'


for genes in files_names:
    genes_list = pd.read_csv('../results/TF_target_TRN/' + genes, sep=',', header = [0])
        
    chrom_repartition = chrom_repartition_hist(genes_list.TG)
    distances = distances_hist(genes_list)
    
    chrom_repartition.write_image("../results/chrom_repartition_hist/" + genes +".jpeg")
    distances.write_image("../results/3Ddistances_hist/" + genes +".jpeg")

### b) Activated targets

In [ ]:
adjacency_matrix = pd.read_parquet("../dashboard/static/adjacency_matrix.parquet.gzip", engine='pyarrow')
files_names = os.listdir("../results/TF_target_TRN_pos")

for genes in files_names:
    genes_list = pd.read_csv('../results/TF_target_TRN_pos/' + genes, sep=',', header = [0])
        
    if len(genes_list.TG) > 0 :    
        chrom_repartition = chrom_repartition_hist(genes_list.TG)
        
        chrom_repartition.write_image("../results/chrom_repartition_hist_pos/" + genes +".jpeg")

# IV- QQ plots

In [ ]:
def distances_QQplot(genes_list, adjacency_matrix, genes):
    
    edges_list = get_edges_list(genes_list, adjacency_matrix)
    x = np.array(edges_list["3D_distances"])
    
    y = []
    for i in range(1, 5) :
        random_genes_list = pd.DataFrame(data = {"Primary_SGDID": rd.choices(adjacency_matrix.index, k=len(edges_list))})
        random_edges_list = get_random_edges_list(random_genes_list, adjacency_matrix)
        y += list(random_edges_list["3D_distances"])
    
    y = np.array(y)
    pp_x = sm.ProbPlot(x)
    pp_y = sm.ProbPlot(y)
    qqplot_2samples(pp_x, pp_y)
    plt.xlabel('genes list 3D distances')
    plt.ylabel('random list 3D distances')
    plt.xlim(0, 200)
    plt.ylim(0, 200)
    plt.plot(x, x, color='k', linestyle='-', linewidth=1)
    genes = genes[:-4]
    plt.savefig("../results/QQplots/" + genes + ".jpeg")
    plt.close()

    return

In [ ]:
adjacency_matrix = pd.read_parquet("../dashboard/static/adjacency_matrix_V4.parquet.gzip", engine='pyarrow')

In [ ]:
for i in range(0, 1000):
    random_genes_list = pd.DataFrame(data = {"Primary_SGDID": rd.sample(list(adjacency_matrix.index), k=10)})
    random_genes_list = random_genes_list.sort_values(by = "Primary_SGDID")
    adjacency_matrix_select = adjacency_matrix.loc[random_genes_list.Primary_SGDID, random_genes_list.Primary_SGDID]
    test = get_random_edges_list(random_genes_list, adjacency_matrix)
    if len(test) == 44 :
        print(len(adjacency_matrix_select.stack().dropna().reset_index()))
        print(len(test))
        display(adjacency_matrix_select)

In [ ]:
adjacency_matrix

In [ ]:
%%time

adjacency_matrix = pd.read_parquet("../dashboard/static/adjacency_matrix.parquet.gzip", engine='pyarrow')
files_names = os.listdir("../results/TF_target_TRN")

for genes in files_names:
    print(genes)
    genes_list = pd.read_csv('../results/TF_target_TRN/' + genes, sep=',', header = [0])
    
    if len(genes_list) < 100 :
        distances_QQplot(genes_list, adjacency_matrix, genes)

# V- 3D distances repartition functions

In [ ]:
def get_edges_list(gene_list, adjacency_matrix):
    
    sql_query = \
"""SELECT Primary_SGDID, Chromosome, Feature_name, Strand, Stop_coordinate, Start_coordinate
FROM SGD_features
"""

    Feature_name = tools.get_locus_info("../SCERE.db", sql_query)
    Feature_name = Feature_name.merge(genes_list, left_on = "Feature_name", right_on = genes_list.columns[0])
    
    adjacency_matrix_select = adjacency_matrix.loc[Feature_name.Primary_SGDID, Feature_name.Primary_SGDID]
    
    edges_list = adjacency_matrix_select.stack().dropna().reset_index()
    edges_list = edges_list.sort_values(by = "Primary_SGDID_bis")
    edges_list.rename(columns = {0: "3D_distances"}, inplace = True)
    edges_list = edges_list.sort_values(by = "3D_distances")
    edges_list.index = range(1, len(edges_list) + 1)
    
    return edges_list

def get_random_edges_list(random_gene_list, adjacency_matrix):
    
    adjacency_matrix_select = adjacency_matrix.loc[random_gene_list.Primary_SGDID, random_gene_list.Primary_SGDID]
    random_edges_list = adjacency_matrix_select.stack().dropna().reset_index()
    random_edges_list.rename(columns = {0: "3D_distances"}, inplace = True)
    random_edges_list = random_edges_list.sort_values(by = "3D_distances")
    random_edges_list.index = range(1, len(random_edges_list) + 1)
    
    return random_edges_list

def distri(genes_list, adjacency_matrix, genes):
    #to copy paste from V2

In [ ]:
adjacency_matrix = pd.read_parquet("../dashboard/static/adjacency_matrix_V4.parquet.gzip", engine='pyarrow')
files_names = os.listdir("../results/TF_target_TRN")

for genes in files_names:
    print(genes)
    genes_list = pd.read_csv('../results/TF_target_TRN/' + genes, sep=',', header = [0])
    fig = distri(genes_list, adjacency_matrix, "YPR104C_287_targets")
    fig.write_image("../results/distri/" + genes + ".jpeg")

# VI- Distributions comparaison

In [ ]:
def ks_2samples(sample, name):
    
    sample_chrom_repartition = chrom_repartition(sample.TG)
    
    random_chrom_repartition = chrom_repartition(random_sample.random_TG)
    
    ks_result = ks_2samp(sample_chrom_repartition, random_chrom_repartition)
    
    return ks_result



def chrom_repartition(genes_list):
    
    sql_query = \
"""SELECT Primary_SGDID, Feature_name, Start_coordinate, Stop_coordinate, Chromosome, Strand
FROM SGD_features
ORDER BY Start_coordinate
"""
    
    loci = tools.get_locus_info("../SCERE.db", sql_query)
    loci = loci.assign(FT_target = loci.Feature_name.isin(genes_list))
       
    loci = loci[loci.FT_target == True].drop(["FT_target"], axis = 1)
       
    return loci.Chromosome

In [ ]:
files_names = os.listdir("../results/TF_target_TRN")
files_names_random = os.listdir("../results/TF_random_target_TRN")


for genes, random_genes in zip(files_names, files_names_random):
    random_genes_list = pd.read_csv('../results/TF_random_target_TRN/' + random_genes, sep=',', header = [0])
    genes_list = pd.read_csv('../results/TF_target_TRN/' + genes, sep=',', header = [0])
    
    ks_2samples(genes_list, random_genes_list, genes)